## Create and update example YAML resources

- read files from local folder
- convert yaml to dict
- create table of elements
    with data from file name
- update resources with table data
- update signatures
- validate
- create include-yaml files
- save files back to folder

In [ ]:
from pathlib import Path
import pandas as pd
from json import dumps, loads
from requests import post
from yaml import load as y_load, dump as y_dump, FullLoader
from IPython.display import display, Markdown, HTML


In [ ]:

fhir_test_server = 'http://test.fhir.org/r4'
fhir_test_server = 'http://hapi.fhir.org/baseR4'
fhir_test_server = 'http://wildfhir4.aegis.net/fhir4-0-1' 

headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
     }
params = {}

includes_path = Path(r'/Users/ehaas/Documents/FHIR/davinci-ecdx/input/includes')
includes_yaml_path = Path(r'/Users/ehaas/Documents/FHIR/davinci-ecdx/input/includes-yaml')
examples_yaml_path = Path(r'/Users/ehaas/Documents/FHIR/davinci-ecdx/input/examples-yaml')
# examples_yaml_out = Path() / 'out' / 'examples-yaml'
# includes_yaml_out = Path() / 'out' / 'includes-yaml'
includes_out = Path() / 'out' / 'includes'
pagecontent_out = Path() / 'out' / 'pagecontent'

In [ ]:
infile = examples_yaml_path / 'cdex-task-example8.yml' #task
data = y_load(infile.read_text(),Loader=FullLoader) #dict
#get DR from task
dr = data['contained'][0]['entry'][0]['resource']
# dr['id']
dr.pop('id')
dr

In [58]:
infile = examples_yaml_path / 'cdex-parameters-example1.yml' #param

param = y_load(infile.read_text(),Loader=FullLoader) #dict
param['parameter'][7]['part'].pop(4)
param['parameter'][7]['part']

[{'name': 'LineItem', 'valueString': '1'},
 {'name': 'LineItem', 'valueString': '2'},
 {'name': 'LineItem', 'valueString': '3'},
 {'name': 'Code',
  'valueCodeableConcept': {'coding': [{'system': 'http://loinc.org',
     'code': '11504-8',
     'display': 'Surgical operation note'}],
   'text': 'Surgical operation note'}}]

In [59]:

param['parameter'][7]['part'].append({"name":"Content","resource": dr})
param['parameter'][7]['part']
# param
infile.write_text(y_dump(param, indent=2, sort_keys=False))


18261

In [63]:
outfile = includes_yaml_path/infile.name
try:
    param.pop('id')
except  KeyError:
    pass
try:
    param.pop('meta')
except  KeyError:
    pass
outfile.write_text(y_dump(param, indent=2, sort_keys=False))


17756

### Write and Validate Functions

In [ ]:

def validate(r): #as dict
    r = post(f'{fhir_test_server}/{r["resourceType"]}/$validate', params = params, headers = headers, data = dumps(r))
    display(Markdown(f'{r.status_code} {r.reason} \n{r.json()["text"]["div"]}'))
    return r.status_code

def write(data,f): #as dict 
    display(Markdown(f'### Writing to {f}....')) 
    #out= Path()/ "test_files" / "test.yml"
    out = Path(f)
    out.write_text(y_dump(data, sort_keys=False)) #as yaml

def write_includes(data,f): # write to include-yaml too  pop off title, description, id
    data.pop('id')
    data['meta'].pop('extension')
    display(Markdown(f'### Writing to {includes_yaml_path / f}....'))
    #out= Path()/ "test_files" / "test.yml"
    out = Path(includes_yaml_path / f)
    out.write_text(y_dump(data, sort_keys=False)) #as yaml   

### Read Examples CSV file

- read master csv file for instructions and update to file names and changes to resources etc.

In [ ]:
my_file = 'cdex-task-examples.csv'
#my_file = 'cdex-parameters-examples.csv'

my_df = pd.read_csv(examples_yaml_path / my_file)
my_df.head(1)

### change file names for examples and includes

- get file - old name, new name
  - create map
- update example id, Title
- save as new example-yaml in temp folder
- strip id and meta and save as new includes-yaml files in temp folder
- update includes file referencing them
- rename includes file
  - create map
- write includes file

### create includes file mapping

In [ ]:
includes_file_map = {}
for index,row in my_df.iterrows():
   old_scenario_file = row.scenario_file
   new_scenario_file = row.scenario_no

   if "|" in old_scenario_file:
      # print(old_scenario_file, new_scenario_file)
      for j,k in enumerate(old_scenario_file.split('|')):
         # print(j, k)
         includes_file_map[k]=f'task-scenario-{new_scenario_file.split("|")[j]}'
   else:
      includes_file_map[old_scenario_file]=f'task-scenario-{new_scenario_file}'
for k in includes_file_map:
   print(k, includes_file_map[k])

In [ ]:
file_name_map = {}
for index,row in my_df.iterrows():
   old_name = row.file_name
   # print(old_name)
   new_name = row.new_file_name
   file_name_map[old_name]=new_name
   #print (file_name_map )
   infile = examples_yaml_path /f'{row.file_name}.yml'
   print(infile)
   data = y_load(infile.read_text(),Loader=FullLoader) #dict
   data['id'] = new_name
   data['meta']['extension'][0]['valueString'] = row.title
   oo = validate(data)
   if 'error' in [i['severity'] for i in oo['issue']]:
      display(Markdown(f'{oo["text"]["div"]}'))
   write(data,f'{examples_yaml_out}/{new_name}.yml')
   data.pop('id')
   data.pop('meta')
   write(data,f'{includes_yaml_out}/{new_name}.yml')
   # print(includes_path)

print (file_name_map )


In [ ]:
for i in includes_path.glob('*scenario*.md'):
   # print (i.stem)
   page = i.read_text()
   # print(old_page.find(old_name))
   for k,v in file_name_map.items():
      # print(k,v)
      if page.find(k) != -1:
         # print(k,v)
         # print(page.find(k) != -1)
         page = page.replace(k,v)
         # print(page)
         # break
   try:
      new_scenario_file_name = includes_file_map[i.stem]
   except KeyError as e:
      pass
   else:
      write(page,f'{includes_out}/{new_scenario_file_name}.md')

In [ ]:
page_path = Path(r'out/pagecontent/task-based-approach.md')
print(includes_file_map)
fin = open(page_path, "rt")
#output file to write the result to
fout = open(pagecontent_out / 'test.md', "wt")
#for each line in the input file
for line in fin:
    for k,v in includes_file_map.items():
    
        #read replace the string and write to output file
        line = line.replace(k, v)
    fout.write(line)   
#close input and output files
fin.close()
fout.close()


### update profile declaration in examples


In [ ]:
for i in examples_yaml_path.glob('*.yml'):
    data = y_load(i.read_text(),Loader=FullLoader) #dict
    if data['resourceType']=="Bundle":
        print(i)
        file_name = i.name
        try:
            profile = data['meta']['profile']
        except KeyError as e:
            profile = "no profile declared"
        print(f'file_name = {file_name}, profile = {profile}')
        new_profile = 'http://hl7.org/fhir/us/davinci-cdex/StructureDefinition/cdex-signature-bundle'
        data['meta']['profile'] = [new_profile]

        print(y_dump(data, indent=2, sort_keys=False)[0:1000])
        v = validate(data)
        # break
        if v == 200:
            write(data,i)
            # break
            # write to include-yaml too
            write_includes(data,i.name)

### update elements:
- Title
- Description
- POU
- WQ

In [ ]:
for index,row in my_df.iterrows():
    my_file = examples_yaml_path / f'{row.new_file_name}.yml'
    data = y_load(my_file.read_text(),Loader=FullLoader) #dict
    # print('resourceType')#=="Task":
    # old_title = data['meta']['extension'][0]['valueString']
    # print(f'old title = {old_title}')
    # new_title = row.new_title
    # print(f'new title = {new_title}')
    # data['meta']['extension'][0]['valueString'] = new_title
    # desc = data['meta']['extension'][1]['valueMarkdown'].split(':')
    # data['meta']['extension'][1]['valueMarkdown'] = f'**{desc[0]}:** {desc[1]}'
    try:
        data['meta']['tag'][0]['code'] = row.work_queue
    except:
        pass
    for input in data['input']:
        input_type = input['type']['coding'][0]['code']
        if input_type == 'purpose-of-use':
            input['valueCodeableConcept']['coding'][0]['code'] = row.POU
            input['valueCodeableConcept']['coding'][0]['system'] = 'http://terminology.hl7.org/CodeSystem/v3-ActReason'
    # break
    write(data,my_file)
# data

In [ ]:
my_df = pd.DataFrame(columns = list(my_df.columns))

my_df

In [ ]:

# new_pou  = '''
# type:
#     coding:
#     - system: http://hl7.org/fhir/us/davinci-cdex/CodeSystem/cdex-temp
#       code: purpose-of-use
# valueCodeableConcept:
#     coding:
#     - system: http://terminology.hl7.org/CodeSystem/v3-ActReason
#       code: COVERAGE
# '''

new_pou  = '''
system: 'http://hl7.org/fhir/us/davinci-cdex/CodeSystem/cdex-temp'
code: operations-nos
'''

# print(new_pou)

# y_load(new_pou)

# new_tag  = '''
# system: 'http://hl7.org/fhir/us/davinci-cdex/CodeSystem/cdex-temp'
# code: claims-processing
# '''
new_tag  = '''
system: 'http://hl7.org/fhir/us/davinci-cdex/CodeSystem/cdex-temp'
code: authorization-other
'''

new_reasonref= '''
#  reference: http://example.org/fhir/Claim/cdex-example-claim
identifier: # ms 0..1 in data profile
    type:  # could add FILL=Payer or PLAC=Provider types
        coding:
            -  system: "http://terminology.hl7.org/CodeSystem/v2-0203"
               code: FILL 
               display: Filler Identifier
        text: Payer Claim ID 
    system: "http://example.org/cdex/payer/claim-ids"
    value: Claim123
'''
new_patientref= '''
#  reference: http://example.org/fhir/Patient/cdex-example-patient
identifier:
        type:
          coding:
            - system: "http://hl7.org/fhir/us/davinci-hrex/CodeSystem/hrex-temp" # vs http://hl7.org/fhir/us/carin-bb/CodeSystem/C4BBIdentifierType - um
              code: UMB
              display: Member Number
          text: Member Number
        system: "http://example.org/cdex/payer/member-ids"
        value: Member123
'''

new_payerref= '''
identifier:
    system: http://example.org/cdex/payer/payer-ids
    value: "Payer123"
'''
new_providerref= '''
identifier:
    system: http://hl7.org/fhir/sid/us-npi #NPI
    value: "9941339108"
'''

new_description = 'foo'

def convert(r):
    r = post(f'{fhir_test_server}/$convert', params = params, headers = headers, data = r)
    print(r.status_code)
    # view  output
    # return (r.json()["text"]["div"])
    return r.json()



In [ ]:


def entitle(name):
    x = name.split('-')
    return(' '.join(i.title() for i in x if i != "example"))

def enname(name):
    x = name.split('-')
    return(' '.join(i for i in x if i != "example"))

def p_name(f):
    #p = loads(f.open().read())
    p=f
    try:
        profile_url = (p['meta']['profile'][0])
        return profile_url.split('/')[-1]
    except KeyError:
        return p['resourceType']

#files = [x for x in Path(in_path).iterdir() if x.is_file() and x.name.endswith('yml')]
#files = [x for x in Path(in_path).iterdir() if x.is_file() and x.name.endswith('xml')] # Get xml and convert and rename to json

for i in examples_yaml_path.glob('*.yml'):
    data = y_load(i.read_text(),Loader=FullLoader) #dict
    if data['resourceType']=="Task":

        file_name = i.name
        print(f'file_name = {file_name}')
        data.pop('id')
        data['meta'].pop('extension')
        data['meta'].pop('profile')
        # break
        write(data,includes_yaml_path / i.name)
# print(y_dump(data, indent=2, sort_keys=False))


In [ ]:
        
        example_id = data['id']
        print(f'example_id= {example_id}') 
        is_same =  i.stem == data['id']
        print(f'is_same = {is_same}')
        title = data['meta']['extension'][0]['valueString']
        print(f'title = {title}') 
        # desc = data['meta']['extension'][1]['valueMarkdown']
        # print(f'desc = {desc}')


        try:
            wq =  data['meta']['tag'][0]['code']
        except KeyError as e:
            wq = None
        else:
            print(f'wq={wq}')
        
        try:
            'identifier' in data['reasonReference']
        except KeyError as e:
            is_identifier = False
        else:
            is_identifier = True
        print (f'identifier = {is_identifier}')
        is_query = False
        pou = None
        loinc = None
        for input in data['input']:
            input_type = input['type']['coding'][0]['code']
            if input_type == 'purpose-of-use':
                pou =input['valueCodeableConcept']['coding'][0]['code']
            elif input_type == 'data-code':
                try:
                   loinc =input['valueCodeableConcept']['coding'][0]['code']
                except KeyError as e:
                   loinc = 'text only!'
            elif input_type == 'data-query':
                is_query = True
        print(f'pou ={pou}')
        print(f'loinc = {loinc}')
        print(f'is_query={is_query}')
        
        my_df.loc[len(my_df.index)] = [file_name, example_id,is_same, title, desc, example_id ,None ,pou , wq,is_identifier, loinc, is_query, None] 

my_df


### save df as csv

In [ ]:
my_df.to_csv(examples_table, index=False)

In [ ]:

    try:
        wq =  data['meta']['tag'][0]['code']
    except KeyError as e:
        pass
    else:
        print(f'wq={wq}')

    try:
      'identifier' in data['reasonReference']
    except KeyError as e:
        print = False
    
    for input in data['input']:
        input_type = input['type']['coding'][0]['code']
        if input_type == 'POU':
            pou =input['valueCodeableConcept']['coding'][0]['code']
            print(f'pou ={pou}')
        elif input_type == 'data-code':
            try:
              loinc =input['valueCodeableConcept']['coding'][0]['code']
              print(loinc)
            except KeyError as e:
                print(f'loinc = text only!')

        elif input_type == 'data-query':
            is_query = True
            print(f'is_query={is_query}')


    




### fill table with existing values

In [ ]:
for i in example_path.glob('*.yml'):

    data = y_load(i.read_text(),Loader=FullLoader) #dict


    my_df.example_id = i.parent

my_df

In [ ]:

    if data['resourceType'] == 'Task':
        #print(f"old tag = { data['meta']['tag'][0]}")
        data['meta']['tag'][0]=y_load(new_tag,Loader=FullLoader)  # update work queue
        #print(f"new tag = { data['meta']['tag'][0]}")
        # update pou
        #print(f"old pou = { data['input'][1]['valueCodeableConcept']['coding'][0]}")
        data['input'][1]['valueCodeableConcept']['coding'][0]=y_load(new_pou,Loader=FullLoader)
        #print(f"new pou= { data['input'][1]['valueCodeableConcept']['coding'][0]}")
        # update reasonref
        #print(f"old reasonref = { data['reasonReference']}")
        data['reasonReference']=y_load(new_reasonref,Loader=FullLoader)
        #print(f"new reasonref= { data['reasonReference']}")
        # update reasonref
        #print(f"old patientref= { data['for']}")
        data['for']=y_load(new_patientref,Loader=FullLoader)
        #print(f"new patientref= { data['for']}")
        # update payerref
        #print(f"old payerref= { data['requester']}")
        data['requester']=y_load(new_payerref,Loader=FullLoader)
        #print(f"new payerref= { data['requester']}")
        # update providerref
        #print(f"old providerref= { data['owner']}")
        data['owner']=y_load(new_providerref,Loader=FullLoader)
        #print(f"new providerref= { data['owner']}")
        #print(y_dump(data, sort_keys=False))
        # validate(data)
        # data['meta']['extension'][1]['valueMarkdown'] += ' This example illustrates the use of business identifiers (instead of pointing to a FHIR resource) to references the various actors.' 
        # print(y_dump(data, sort_keys=False)) 
        # break
        write(data,i)